# Processing multiple interferograms with ISCE

In [1]:
# Import required packages
import logging
log = logging.getLogger()
log.setLevel(logging.WARN)
import os
import getpass
import asf_search as asf
import isce
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mc
from osgeo import gdal
from datetime import datetime, timedelta
import xarray as xr
import rasterio as rio
import rioxarray
import geopandas as gpd
import time

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-12-18 21:33:26,464 - matplotlib - DEBUG - matplotlib data path: /home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/matplotlib/mpl-data
2022-12-18 21:33:26,471 - matplotlib - DEBUG - CONFIGDIR=/home/jovyan/.config/matplotlib
2022-12-18 21:33:26,473 - matplotlib - DEBUG - interactive is False
2022-12-18 21:33:26,474 - matplotlib - DEBUG - platform is linux
2022-12-18 21:33:26,544 - matplotlib - DEBUG - CACHEDIR=/home/jovyan/.cache/matplotlib
2022-12-18 21:33:26,547 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/jovyan/.cache/m

In [2]:
# Set environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)
print(os.environ['PATH'])

/home/jovyan/.local/ARIA-tools/tools/bin:/home/jovyan/.local/ARIA-tools/tools/ARIAtools:/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications:/home/jovyan/.local/envs/insar_analysis/bin:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/bin:/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications


In [3]:
!which topsApp.py

/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications/topsApp.py


In [ ]:
scene_list = ['S1A_IW_SLC__1SSV_20160806T010216_20160806T010242_012473_0137EC_EFCE',
              'S1A_IW_SLC__1SDV_20160830T010216_20160830T010243_012823_01439D_CA21',
              'S1B_IW_SLC__1SDV_20170726T010149_20170726T010216_006652_00BB37_781F',
              'S1B_IW_SLC__1SDV_20170807T010150_20170807T010217_006827_00C043_4F24',
              'S1B_IW_SLC__1SDV_20170819T010151_20170819T010218_007002_00C561_A180',
              'S1B_IW_SLC__1SDV_20170912T010152_20170912T010219_007352_00CF90_41DC',
              'S1B_IW_SLC__1SDV_20180721T010156_20180721T010223_011902_015E85_3A3A',
              'S1B_IW_SLC__1SDV_20180802T010156_20180802T010223_012077_0163CE_DC97',
              'S1B_IW_SLC__1SDV_20180814T010157_20180814T010224_012252_016935_51BF',
              'S1B_IW_SLC__1SDV_20180826T010158_20180826T010225_012427_016EA3_9F1F',
              'S1B_IW_SLC__1SDV_20180907T010158_20180907T010225_012602_017408_FF2F',
              'S1B_IW_SLC__1SDV_20190716T010202_20190716T010229_017152_020442_F860',
              'S1B_IW_SLC__1SDV_20190728T010202_20190728T010229_017327_020959_B45F',
              'S1B_IW_SLC__1SDV_20190809T010203_20190809T010230_017502_020EA0_DD3F',
              'S1B_IW_SLC__1SDV_20190821T010204_20190821T010231_017677_021418_5589',
              'S1B_IW_SLC__1SDV_20190902T010204_20190902T010231_017852_021986_3E3E',
              'S1B_IW_SLC__1SDV_20190914T010205_20190914T010232_018027_021EFB_AF4A',
              'S1B_IW_SLC__1SDV_20200722T010208_20200722T010235_022577_02AD97_AE1D',
              'S1B_IW_SLC__1SDV_20200803T010209_20200803T010236_022752_02B2E3_C178',
              'S1B_IW_SLC__1SDV_20200815T010210_20200815T010237_022927_02B84B_FCB6',
              'S1B_IW_SLC__1SDV_20200827T010211_20200827T010238_023102_02BDCC_BB77',
              'S1B_IW_SLC__1SDV_20200908T010211_20200908T010238_023277_02C345_0255',
              'S1B_IW_SLC__1SDV_20210717T010214_20210717T010241_027827_035208_A4DF',
              'S1B_IW_SLC__1SDV_20210729T010215_20210729T010242_028002_03572B_9913',
              'S1B_IW_SLC__1SDV_20210810T010215_20210810T010242_028177_035C89_6988',
              'S1B_IW_SLC__1SDV_20210822T010216_20210822T010243_028352_036202_417A',
              'S1B_IW_SLC__1SDV_20210903T010217_20210903T010244_028527_036777_F591'
             ]

In [ ]:
def select_pairs(scene_list, max_temp_bline):
    scene_dates = {}
    for scene in scene_list:
        date = scene[17:25]
        scene_dates[date] = scene
        
    pair_dict = {}
    pair_scenes = []
    for date1 in scene_dates:
        for date2 in scene_dates:
            if datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=max_temp_bline) and not date1 >= date2 :
                pair_dict[f'{date1}-{date2}'] = [scene_dates[date1], scene_dates[date2]]
                pair_scenes.append(scene_dates[date1])
                pair_scenes.append(scene_dates[date2])
    pair_scenes = [*set(pair_scenes)]
    
    print(f'number of pairs: {len(pair_dict)}')
    
    return pair_dict, pair_scenes

In [ ]:
pair_dict, pair_scenes = select_pairs(scene_list, 40)

In [ ]:
proc_path = '/home/jovyan/rmnp_landslide/proc_T151A'
dem_name = 'usgs_10m.dem.wgs84'

In [ ]:
for pair in pair_dict:
    pair_path = f'{proc_path}/{pair}'
    if not os.path.exists(pair_path):
                          os.makedirs(pair_path)
    !cp '{proc_path}/{dem_name}.xml' '{pair_path}/{dem_name}.xml'
    !cp '{proc_path}/{dem_name}' '{pair_path}/{dem_name}'

## Download all SLCs

In [ ]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

In [ ]:
# Change to SLC directory
os.chdir(f'{proc_path}/slc')

logging.getLogger("urllib3").setLevel(logging.WARNING)

results = asf.granule_search(pair_scenes)
session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path=f'{proc_path}/slc', processes=2, session=session)

## Download orbital files

In [ ]:
os.chdir(f'{proc_path}/orbital')

In [ ]:
%%bash

wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

In [ ]:
# grab orbital files with fetchOrbit.py
for scene in pair_scenes:
    os.system(f'./fetchOrbit.py -i {scene}')

## Write input files

In [ ]:
for pair in pair_dict:
    os.chdir(f'{proc_path}/{pair}')
    
    reference = pair_dict[pair][0]
    secondary = pair_dict[pair][1]
    
    with open('topsApp.xml', 'w') as f:
        f.write(f"""<?xml version="1.0" encoding="UTF-8"?>
    <topsApp>
      <component name="topsinsar">
        <property name="Sensor name">SENTINEL1</property>
        <component name="reference">
            <property name="orbit directory">{proc_path}/orbital</property>
            <property name="output directory">reference</property>
            <property name="safe">{proc_path}/slc/{reference}.zip</property>
        </component>
        <component name="secondary">
            <property name="orbit directory">{proc_path}/orbital</property>
            <property name="output directory">secondary</property>
            <property name="safe">{proc_path}/slc/{secondary}.zip</property>
        </component>
        <property name="demfilename">{proc_path}/usgs_10m.dem.wgs84</property>
        <property name="useGPU">True</property>
        <property name="range looks">6</property>
        <property name="azimuth looks">1</property>
        <property name="swaths">[1]</property>
        <property name="region of interest">[40.295, 40.310, -105.69, -105.67]</property>
        <property name="do esd">False</property>
        <property name="do ionospherecorrection">False</property>
        <property name="do unwrap">True</property>
        <property name="unwrapper name">snaphu_mcf</property>
        <property name="do denseoffsets">False</property>
        <property name="geocode demfilename">{proc_path}/usgs_10m.dem.wgs84</property>
        <property name="geocode list">['merged/topophase.cor', 'merged/filt_topophase.unw', 'merged/filt_topophase.flat']</property>
      </component>
    </topsApp>""")

## TopsApp processing

In [ ]:
%%time

for pair in pair_dict:
    os.chdir(f'{proc_path}/{pair}')
    
    !topsApp.py --start=preprocess --end=geocode

## Filter and stack interferograms
### Load igrams into xarray

In [ ]:
# functions to load interferogram tifs to xarray

def xr_read_vrt(vrt_file_path, variable_name, masked=True):

    da = rioxarray.open_rasterio(vrt_file_path, masked=True)

    # Extract phase and assign as variable in xr.Dataset()
    ds = xr.Dataset()
    da_phase = da.sel(band=2)
    da_phase.name = variable_name
    ds[da_phase.name] = da_phase

    # Preserve top-level attributes and extract single value from value iterables e.g. (1,) --> 1
    ds.attrs = da.attrs
    for key, value in ds.attrs.items():
        try:
            if len(value) == 1:
                ds.attrs[key] = value[0]
        except TypeError:
            pass
    
    # crop to roi
    ds = ds.rio.clip_box(minx=-105.69, miny=40.29, maxx=-105.66, maxy=40.31) 

    return ds

In [ ]:
igram_dss = []
coh_dss = []
igram_name = 'merged/filt_topophase.unw.geo.vrt'
coh_name = 'merged/topophase.cor.geo.vrt'

for pair in pair_dict:
    igram_src = xr_read_vrt(f'{proc_path}/{pair}/{igram_name}', 'unw_phase')
    igram_src = igram_src.assign_coords({"dates": pair})
    igram_src = igram_src.expand_dims("dates")
    igram_dss.append(igram_src)
                            
    coh_src = xr_read_vrt(f'{proc_path}/{pair}/{coh_name}', 'coherence')
    coh_src = coh_src.assign_coords({"dates": pair})
    coh_src = coh_src.expand_dims("dates")
    coh_dss.append(coh_src)

In [ ]:
# standardize coordinates before concat
igram_dss_standard = [igram_dss[0]]
coh_dss_standard = []

x_reference = igram_dss[0]['x'].values
y_reference = igram_dss[0]['y'].values

for ds in igram_dss[1:]:
    ds = ds.assign_coords({'x': ('x', x_reference),
                      'y': ('y', y_reference)})
    igram_dss_standard.append(ds)
    
for ds in coh_dss:
    ds = ds.assign_coords({'x': ('x', x_reference),
                      'y': ('y', y_reference)})
    coh_dss_standard.append(ds)
    

igram_ds = xr.concat(igram_dss_standard, dim='dates', combine_attrs="no_conflicts") #create dataset
coh_ds = xr.concat(coh_dss_standard, dim='dates', combine_attrs="no_conflicts") #create dataset
igram_ds['coherence'] = (('dates', 'y', 'x'), coh_ds['coherence'].values)

### Change reference points

In [ ]:
# function to find value of new reference point in each time slice and subtract it from data array
rref_list = []

def change_ref(ds, lat, lon):
    reref = ds.copy(deep=True)
    for i in range(ds.sizes['dates']):
        # get value from grid
        rref_list.append(float(ds.isel(dates=i).unw_phase.sel(x=lon, y=lat, method='nearest').values))
    reref['unw_phase'] = ds['unw_phase'] - xr.DataArray(rref_list, dims='dates')
    return reref 

In [ ]:
igram_ds = change_ref(igram_ds, 40.302247, -105.672370)

### Convert to velocities in m/yr

In [ ]:
delta_list = []
for date in igram_ds['dates'].values:
    delta_list.append((datetime.strptime(date[9:17], '%Y%m%d')-datetime.strptime(date[0:8], '%Y%m%d')).days)
    
igram_ds = igram_ds.assign_coords({'timedelta':('timedelta', delta_list)})

veloc = (((igram_ds['unw_phase']*0.05546576/12.5663706)/igram_ds['timedelta']))[0].values
igram_ds['veloc'] = (('y', 'x', 'dates'), veloc.data)
igram_ds = igram_ds.drop_dims('timedelta')

In [ ]:
f, ax = plt.subplots()
ax.imshow(igram_ds['veloc'].median(dim="dates"), cmap='RdBu', vmin=-0.3, vmax=0.3) 

### filter and stack

In [ ]:
# all years 
f, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].imshow(igram_ds['veloc'].where(igram_ds['coherence'] > 0.5).mean(dim='dates'),
         cmap='RdBu', vmin=-0.4, vmax=0.4)
ax[1].imshow(igram_ds['veloc'].where(igram_ds['coherence'] > 0.5).count(dim='dates'), cmap='Blues', vmax=43)
f.tight_layout()
ax[2].imshow(igram_ds['veloc'].where(igram_ds['coherence'] > 0.5).std(dim='dates'), cmap='Blues', vmax=0.1)
f.tight_layout()

In [ ]:
# save filtered velocities and counts 
#igram_ds['veloc'].where(igram_ds['coherence'] > 0.5).mean(dim='dates').rio.to_raster('asc_veloc_mean_0.5coh.tif')

# for some reason these two aren't saving correctly. 
igram_ds['veloc'].where(igram_ds['coherence'] > 0.5).count(dim='dates').rio.to_raster('asc_veloc_count_0.5coh.tif')
igram_ds['veloc'].where(igram_ds['coherence'] > 0.5).count(dim='dates').rio.to_raster('asc_veloc_std_0.5coh.tif')

### mean veloc in feature

In [ ]:
# load json of aoi 
aoi_fn = '/home/jovyan/rmnp_landslide/moving_area.shp'
aoi_gdf = gpd.read_file(aoi_fn)

In [ ]:
igram_aoi = igram_ds.rio.clip(aoi_gdf.geometry, crs=aoi_gdf.crs, drop=False)

In [ ]:
# mean velocity
np.nanmean(igram_aoi['veloc'].where(igram_ds['coherence'] > 0.5).values.ravel())

In [ ]:
# velocity std dev
np.nanstd(igram_aoi['veloc'].where(igram_ds['coherence'] > 0.5).values.ravel())

In [ ]:
# max velocity
np.nanmin(igram_aoi['veloc'].where(igram_ds['coherence'] > 0.5).mean(dim='dates').values.ravel())

In [ ]:
# mean coherence
np.nanmean(igram_aoi['coherence'].values.ravel())

In [ ]:
# average number of ints used to calculate velocities
temp = igram_aoi['veloc'].where(igram_ds['coherence'] > 0.5).count(dim='dates')
np.nanmean(temp.where(temp != 0))

## Make igram baseline figure

In [ ]:
def toYearFraction(date):
    def sinceEpoch(date): # returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch

    year = date.year
    startOfThisYear = datetime(year=year, month=1, day=1)
    startOfNextYear = datetime(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

In [ ]:
lines = []
for i, date in enumerate(igram_ds['dates'].values):
    lines.append([(toYearFraction(datetime.strptime(date[0:8], '%Y%m%d')), i), (toYearFraction(datetime.strptime(date[9:17], '%Y%m%d')), i)])

In [ ]:
lc = mc.LineCollection(lines, linewidths=4)

plt.style.use('ggplot')
f, ax = plt.subplots(figsize=(5, 5))
ax.add_collection(lc)
ax.margins(0.1)
ax.set_ylabel('index')

plt.savefig('tbline.png', dpi=300)

In [ ]:
# interferogram length capped at 40 days to avoid decorrelation
# mean interferogram length: 21.8 days, max length: 36 days, min length: 12 days